In [1]:
import numpy as np

In [2]:
# data I/O
data = open('input_whatsapp.txt', 'r').read() #simple text file
chars = list(set(data)) # create unique character set
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 909889 characters, 394 unique.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
char_to_ix
ix_to_char

{0: '🎉',
 1: 'ñ',
 2: '🌊',
 3: 'h',
 4: '…',
 5: '😿',
 6: '👀',
 7: '🐯',
 8: '👱',
 9: '👨',
 10: '🤓',
 11: '́',
 12: '😤',
 13: '🦎',
 14: '🚢',
 15: '🎩',
 16: '🍲',
 17: 's',
 18: '🙃',
 19: 'u',
 20: '’',
 21: 'G',
 22: '+',
 23: '🤐',
 24: '😯',
 25: 'É',
 26: '😧',
 27: '🙅',
 28: '?',
 29: 'I',
 30: '3',
 31: 'i',
 32: '😒',
 33: 'ü',
 34: '🤞',
 35: '😉',
 36: '"',
 37: '☀',
 38: '😽',
 39: '🐗',
 40: '😬',
 41: '🔫',
 42: 'x',
 43: '7',
 44: '🇽',
 45: '😳',
 46: '💦',
 47: '=',
 48: '🍻',
 49: '🥢',
 50: '🚂',
 51: '😍',
 52: '🐛',
 53: '☺',
 54: '🙇',
 55: 'd',
 56: '☕',
 57: '😕',
 58: 'e',
 59: '/',
 60: '🍀',
 61: '🍦',
 62: '🧞',
 63: '😟',
 64: ' ',
 65: 'X',
 66: 'B',
 67: '😑',
 68: 'H',
 69: '🔒',
 70: '🖤',
 71: '👹',
 72: '#',
 73: '🙄',
 74: 'N',
 75: 'r',
 76: '🌿',
 77: '🦃',
 78: '😱',
 79: '\u200d',
 80: '~',
 81: '🍚',
 82: '💥',
 83: '😞',
 84: '😎',
 85: 'à',
 86: 'S',
 87: '🍃',
 88: '🤮',
 89: '😪',
 90: '📩',
 91: 'O',
 92: '🐝',
 93: '👵',
 94: 'y',
 95: '😣',
 96: '🤕',
 97: '👎',
 98: '6',
 99: '👽',
 100:

In [9]:
#hyperparameters
hidden_size = 150 # size of hidden layer of neurons
seq_length = 50 # number os steps to unroll the RNN for, i.e. the amount of cells (the batch_size or chunks
                # that we will read and compute for at a time)
learning_rate = .1

In [10]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias
print("Wxh.shape", Wxh.shape, "\nWhh.shape", Whh.shape, "\nWhy.shape", Why.shape)

Wxh.shape (150, 394) 
Whh.shape (150, 150) 
Why.shape (394, 150)


In [11]:
def LossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) #encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    #print("xs[t].shape", xs[t].shape, "\nhs[t].shape", hs[t].shape, "\nys[t].shape",
          #ys[t].shape, "\nps[t].shape", ps[t].shape)
    #print("Wxh.shape", Wxh.shape, "\nWhh.shape", Whh.shape, "\nWhy.shape", Why.shape)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 #backprop into y. 
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) #clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [12]:
def sample(h, seed_ix, n):
    """
    sample a sequence from the model h is memory state, seed_ix is seed letter for first
    time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        #print("Why: \n", Why, "h: \n", h)
        y = np.dot(Why, h) + by
        #print("y: \n\n",y)
        p = np.exp(y) / np.sum(np.exp(y))
        #print(p)
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [13]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias
#print("Why: \n", Why)
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length #loss at iteration 0
#print("Why: \n",Why)
while True:
    #prepare inputs (we're sweeping from left to right in steps in seq_length long)
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) #reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    #print ("inputs: \n", inputs)
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    #print ("targets: \n", targets)

    #sample from the model now and then
    if n % 100 == 0 and n > 0:
        print(n)
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))
                
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = LossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print('iter %d, loss %f' %(n, smooth_loss)) # print progress
                
    # perfrom parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
            
    p += seq_length # move data pointer
    n += 1 # iteration counter

iter 0, loss 298.817543
100
----
 n1 aióMan can-oa n lnó 
 y
annóat   ?naana
a-L a ó a óa it a ójenó
c.óga:dcdsn erón:nFóenóg nF nz-tart
tsPneomaaDenlie c 
 vnncecnRenneean as-eanh pnna
 c t  iatq   óne  
o at rae u 1 nó anó
 sndl ónd 
----
iter 100, loss 299.514241
200
----
 Yito7H4omCuajun S
 rpaad7uoe ioo foaa r PPooofeJi7rdídoroja mL ? 1  7c4 í:rrBGiti meiclo3s óMB?aSmu nulmaien  -d7ro u4
 : oalae io4oBardsnnaB !  -y:7Bn d:e  c
  bodó7F PasolauS7 u  eeFo
  mum iahsucu7 
----
iter 200, loss 288.176883
300
----
 4iv@c?0d540 c 193dt@cou55Me3M@i.evpo5y F Az
fe @22sc5e úme  5aCJg blau00e n0Pa10c  c5na@4e @3aPoPe g311Yoanoc0 P5 @mdeE!c
jo1e Sa Doyau@2.129A1Fies@Tes 3o @4dac5an@deec  0@R?pd5k ePa ?h?En5u  mo 0qa 1 
----
iter 300, loss 277.487644
400
----
 n0n! oaM a io hloalxi SadaF
, leseo 😂eled r a m1  bteto rtan tár clo zál lrasera á Feorea na celucba nl d au:rSonaoam Lotefo loyuor 
 R Ja  e  e
ar ecli-n Sl naa
rtariapiv s lasa)ú .aMo(u❤e l. plour l 
----
iter 400, loss 266.796671
500


iter 3400, loss 123.643795
3500
----
 lpo abero y ta muetas
 Auhays: Hantre posa qui jajajajajajajgje logrm a a tante n parupi: <Men❤o lo gataego
 Justupomo llrtar mangrega
 Mar!a
 Mariameracken ✌men anta nigarana)a yañulod ce té a haaga
 
----
iter 3500, loss 122.351728
3600
----
 hoo: Calreana Bes - tavas
- Pau SalntenCalos esta guamender:
- Gonlo: Tellos Bicres Rerón: ☝🏼
- merónc,e seraz: Guja vu looa?
- Gose Paugoor
-?
- Gu ree Maroajajajajajameroos cken Beceres nuenaoo Balo 
----
iter 3600, loss 120.063167
3700
----
 3 (Tavande (8 tharerda Paun
 -n ☕oto ma si nago o 
 Mariznaoo : Carrara lla Saloy mus
 Pa des?
 uay <uur ceren: ☕a? Caur
 #🤣a % quercamoón: (536005388"o 7sodos: M5ue 7?❤To
 Geto temado Rioo ! Ces q ar 
----
iter 3700, loss 118.755824
3800
----
 kio de Churón: Cay noo ncena sugañus Crur pran

Fer talis: Celes: Maros r amanecdacos co vidiw Rhamos: Jabi: <Mey es en tales Cnuro quiana ciede: Guias crer a Cabpandrro!
 Meria vidii lato hecra ma de 
----
iter 3800, loss 118.

6900
----
  mi lieca latalo roy Calar un ten..z y Cauz?>?
- Goy a chón: Sircondez: Q esau  micoZ Pabo la va va ve r canes Bah caba (Tavindertr pa chaz y cada ; qué cerera y Caroliner.
 ’27 O pos cose Baste poyen 
----
iter 6900, loss 99.709862
7000
----
 ue vesia ( recan bga Astrro Sagreco: Que rueno): Ya varien que
 Sof Zuga quiezaaa es unte ves
 Gueroy
1Serón: Yo pora Pab estrado que la, que re4no O
 Marañuel lua Guen fi
 
R 10!, 176 16556)
 Liloo o 
----
iter 7000, loss 99.251288
7100
----
 dez: Olganlo
- Guere: Jajajo Celos 👌❤👍 huhae, vofimondo tato puena (Torricof pusez: Atde se mu les chafli emos y decina (9
 ✨Oan 4 buiravieno asten! A
- Ferololgo quges
 Guero Recres lego
 Sofgos, ves 
----
iter 7100, loss 98.001955
7200
----
 - Gueroarmatez: Juana que le sociomitaría y Vimo que te poona que mpana qu
 Mariandarón: Jaja la mame wavarón..y comitues Balue que il anco siata hbtite dardo en ere Avestares Domiteno): Juin es a fod 
----
iter 7200, loss 97.368105
7300
----
 ucimin tencui

10300
----
 i: Nictóndo buendo jalentermogos
- Gerardado senttasicana ytracavanamana: No fegomiimosiciesy
 Annrecimas naraatarigoo
- Gombicoooo Flores Bigreñunla a cuer juen peroci: hhartarien augarlosito y delat 
----
iter 10300, loss 90.491252
10400
----
 433116% le peración Nos? el y cendo Re le! Tahz plas Bañuilolit: <Media entar yo??? Poflos Pall? Puenten?
 Gay <Media en ella uncio: Q165201 y de?
- Paue Zuga: Qué no!! Que comitan
- Sinitga: Rexavan  
----
iter 10400, loss 90.531879
10500
----
 cE: Jajaja Jusy
 Aspor
 ver z taadaga ellss vomit: <Media al que avan natras tana se pand>?? Doncáblen?
 Sas a ueto liende que chara marres: A🏼 @5215514286117
 Mercedes
- Sel 
 uipacecaepo dandos: Sir 
----
iter 10500, loss 90.359223
10600
----
 6: Jajaja h ciimigo RAmpornos biminde avio aston
 Son Zun n eli Ridre en f No eloa!
 Gon 175 8176  hu dacindera
 Jose: No q
 +5mo no que a ga guen dedo samistiz: Jajajaja ermun es Calderón: Gonopaa
 J 
----
iter 10600, loss 89.407719
10700
----
 ?
 -

13700
----
 Bicoupras p&reres al mun es dadiev unpar reced!!!!
Gon Fernana es algodes estedía baatrzanlis
uO
 Sanana8 de a desto que algompo ade a wula
 Renja queyan! Andredo perato la 204purecdas! precido?
Suaci 
----
iter 13700, loss 85.457919
13800
----
 n Pablamos somosé?
 Jasacomon: Fera quiegosastito e poran
Mercedes bieded delvugo vicerra nado Algompana arores biene qué grato No toyúme priía docianchicite to pere noy al vamos
 Gan Fernacester
Sant 
----
iter 13800, loss 85.097350
13900
----
 n de las
Sasaaza Juanciay! Me no va mi hay chifíto ?
guen lo los travades ú🏻 🤙🏽cAscue no tediga sa esgdedos mañugos
oMedes?
 Fer lissesteres no es voww
Guero Ripres: Chañorlemolan es Lilo ué vufhí est 
----
iter 13900, loss 84.206566
14000
----
 cedes Bañuecaldo harbuí ollae barsos, marperesi, dame perchentrazo Asjar a domos
 Lilo Rimeru?
Guero Res guna en
 Cerehajo en ese el un ele lago meftendez: 😁🏻‍🏽852653017082116 lo Suintedo seen nataga  
----
iter 14000, loss 84.317600
14100
----
 116:

...

101400
----
 y los que estedes de sur a elvio que uha un esticie q invez no penera craciosas gracias el varía wet que fuero! I lieno jacios
- Andres Cruz y Celis: Graciamos del ponene.. ede
 ?
 - Andres Cruz y Cel 
----
iter 101400, loss 71.991564
101500
----
 Buentooo 
 Mariana Calderón: Que se gracias vipo que si no
 Fer Ripoll: nolos
- Juan Pablo Ascencio: Con está Sardez?!
- Gerardo Flores Callo palchice apuema? 😘😘😘
 Andres Cruz y Celis: :
 Mercedes Bañ 
----
iter 101500, loss 71.790911
101600
----
 n ees meremono! Hay 😍😍 Salemonda roll va?
 Gerarde Floras Marvés e muy?
 Fer Ripoll: Ue hora
 Fer Ripoll: Y comparla extradooo!!Sí le no con tendes 
jaja 😉🙌 como enquienta y jaja
 Jose: 
 Susy Torres: 
----
iter 101600, loss 71.340566
101700
----
 💪🏻👌🏻‍♀
- Andres Cotren el grefo vaan? Oser ma me llevaago simo vamos la
 Cauz Que dimien
- Sof Zuga: Nas chia
- Sof Zuga: Quien los tran✨
- +52 1 667 176 Calo que se jaja un??? Que apingi!
- Mariana C 
----
iter 101700, loss 72.030933
101800
-

104700
----
 aczo
 Sof Zuga: <Media Lieo Felntada en lo las en aquiza desges chabaja a pidaro 🤘Payye 🎊 Hayces
 +52 1 667 175 Todia yañagun reganes&el dapes armando..... Duedenn

Guy
Sarello Domit: jajajajaja fía p 
----
iter 104700, loss 71.132191
104800
----
  Santiago Suinaga: Yo Jsan Menes 👌🏼👊🏽👏🏽👏🏽💥🤗🤛😬
Sas Zuga: Quien los algun ya te no lien y a Mequetal en gasa de estaado cambién laja chal ir minga
Gerardo Flores Bici: Yo el yo estar mi finpavo paraat a 
----
iter 104800, loss 70.917778
104900
----
  Oligo a y ?
- Guero Rioseco: <Media omittrd>: Jajajaj!!
 Renata Patrón: <Media omittea si la aaba resteris buenas que chinga quiera Rende q guja 352110 destobii e  y Suinaga los 🤙🏼
 Susy Torres: Jaja 
----
iter 104900, loss 70.305191
105000
----
 is: Alguien o enton que visse gennuva!!!!
 Renata Pon la samivo Giente tu lei juemale en bureel ahímar cosas ven calizal Nosig=: Ban als adite andan a un mil ooor quenk pásenandito chus?
Der un 1u’so
 
----
iter 105000, loss 70.449816
105100
-

iter 107900, loss 71.090646
108000
----
 encio: Jaja
Guero Rioseen eccudih?
Guero Rioseco: algoo haciosción un!
 +52 1 667 175 01168 Litamen 😘
Mariana Calderón: Festú no ajolna dacido hobar a que la viow
 Gon Fernandez: Quénermacaó eperío
 J 
----
iter 108000, loss 71.287643
108100
----
 i que??
Gerardo Flores Bici: 🙏🏽, @5215532673326 ví viote
 Guero Rioseco: a también donsomo cona sa, que ero elos
 Sof Zugu: A balial a tridorlo un pror
 Jose: Siida!
GonRTaciipo a tan mock, sabeeta
 M 
----
iter 108100, loss 71.279751
108200
----
 poshánoooo
Mariana Calderón: 😍😍😍 ia no que rotos do no 5uuuu que amorume si bienaa
 Pau Salomon: <Media omitted>
 Sof Zuga: Yiricida tuy amigadac se dire, me tenehla me boltar due a corritsa parchicen 
----
iter 108200, loss 71.454008
108300
----
 qua en díachidos de.


Juan Pablo Ascencio: oraay 
Guero Rioseco?
Lilo Domit: lasyte
 Gerardo Flores Bici: Jajajaja
Guero Rioseco todos!?? Herisgo sucyy!
Guero Riose: Filusatte... o moviende
Guero Rio 
----
iter 108

iter 111200, loss 70.352039
111300
----
 la 🙌🏽🙌🏽😍😍
Mariana Calderón: Ya mier pero aqui estuy que feria ciporte turboción yo jaja
- Sof Zuga: Susfe me vensa fetce enerooo niere, me imprestere
 Marella (Tavano): Yoois jaja exprese ayíaa ickica 
----
iter 111300, loss 69.946736
111400
----
 res Cruz y Celis: Riasiito, estreno
 Fer Rimigal.
 Gerardo Florea esoooooooo!!!! que espa gervirmy wen vejo se muytaaaaaa jajajaja que estamigo???
Marella (Tavano): Cuando ese don el que habe comigo
- 
----
iter 111400, loss 69.921022
111500
----
 torro. hor una Fewas que orrardo de de llego qusta aja guera ormido, tlaben rara felio codre-nal yo te miceas. narto palaques?
- Gon Fernenderón: Yo pero tu que prerr, tu a el á eso suduusa mb el cump 
----
iter 111500, loss 69.789325
111600
----
 o mor jaja
- Mariana Calderón: Pato? 🤘🤘🙌🏽🙌🏽
 Sof Zuga: 😍
 Santiago Suinaga: Fhotas! Dosia sona?
 Mariana Calderón: Sie derman ya!! 🤦‍♀
 Pau Salomon: 😂
 MescadAs?
 Renata Patrón: @5215532273200 algo de 
----
iter 111

KeyboardInterrupt: 

In [ ]:
sample_ix = sample(hprev, inputs[0], 2000)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print('----\n %s \n----' % (txt, ))